<a href="https://colab.research.google.com/github/gabrielborja/machine_learning/blob/main/decision_trees_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Decision Trees for Classification

## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

## 01 Data Preprocessing Classification

In [ ]:
# Loading first dataset from local drive
from google.colab import files
uploaded_1 = files.upload()

In [ ]:
# Loading second dataset from local drive
from google.colab import files
uploaded_2 = files.upload()

In [ ]:
# Storing dataset 1 to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded_1['0731_chu.xlsx']), sheet_name='Sheet1')

In [ ]:
# Storing dataset 2 to a pandas dataframe
import io
df2 = pd.read_csv(io.BytesIO(uploaded_2['0731_sto.csv']), sep=',')

In [ ]:
# Modifying columns to lower case for both datasets
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)
df2.rename(columns={i:i.lower() for i in df2.columns}, inplace=True)

In [ ]:
# Dropping unneccesary columns from both datasets
df1.drop(columns=['company_churn'], axis=1, inplace=True)
df2.drop(columns=['company_churn'], axis=1, inplace=True)

In [ ]:
# Sampling dataset 2 to balance both classes
bl_size = int((df1.shape[0])*1.30)
#bl_size
df3 = df2.sample(n=bl_size, axis=0, replace=False, random_state=0)

In [ ]:
# Stacking both dataframes containg both classes
df_cl = pd.concat([df1, df3], ignore_index=True)

In [ ]:
# Checking the dataframe
df_cl.tail()

In [ ]:
# Checking the dataframe info
df_cl.info()

In [ ]:
# Creating the matrix of features and the dependent variable vector
X = df1.iloc[:,:-1].values
y = df1.iloc[:,-1:].values

In [ ]:
# Splitting the dataset into the training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0, shuffle=True)

## 02 Building the Decision Tree for Classification